In [1]:
import joblib
from tensorflow.keras.models import load_model
import tensorflow as tf
import pandas as pd
import numpy as np

C:\Users\rutuj\AppData\Roaming\Python\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
df=joblib.load('../saved_objects/sample_db.joblib')

print("Original:", df.shape)
home = [1, 2]
month = [1]

new_df = df[df['home_id'].isin(home) & df['month_of_year'].isin(month)].sort_values(['home_id', 'timestamp', 'device_type'])
print("After month filter:", new_df.shape)

Original: (1752000, 22)
After month filter: (29760, 22)


In [3]:
non_num_cat = ['device_type', 'room', 'activity']
binary_cat = ['status', 'is_weekend']  # turn to 0s and 1s
num_cols = ['indoor_temp', 'outdoor_temp', 'humidity', 'light_level']
other_cols = ['home_id', 'device_type', 'user_present', 'timestamp', 'day_of_week', 'hour_of_day', 'month_of_year', 'energy_lag_1H', 'energy_lag_1D',
       'energy_lag_1W', 'energy_roll_mean_1hr', 'energy_roll_mean_12hr',
       'energy_roll_mean_24hr']

# separating numeric columns to scale after splitting
X_num = new_df[num_cols].copy()

# Columns that do not need to undergo transformation
X_rem = new_df[other_cols].copy()

X_bin = new_df[binary_cat].copy()
# turn to 0s and 1s
X_bin['status'] =  X_bin['status'].map({'off': 0, 'on': 1})    
X_bin['is_weekend'] =  X_bin['is_weekend'].astype(int)   

# Concatenating all features after transformation to form independent features
X = pd.concat([X_rem, X_bin, X_num], axis=1)

encoder = joblib.load('../saved_objects/ohe.joblib')    # loading saved encoder from milestone 1

# Encoding labeled data
X_cat_array = encoder.transform(new_df[non_num_cat])
encoded_cols = encoder.get_feature_names_out(non_num_cat)
X_cat = pd.DataFrame(X_cat_array, columns=encoded_cols, index=new_df.index)

X = pd.concat([X, X_cat], axis=1)

final_col_order = ['home_id','device_type', 'timestamp', 'user_present', 'status', 'is_weekend', 'hour_of_day','day_of_week', 'month_of_year', 'indoor_temp', 'outdoor_temp', 'humidity', 'light_level', 'device_type_air_conditioner', 'device_type_fridge', 'device_type_light', 'device_type_tv', 'device_type_washer', 'room_bedroom', 'room_kitchen', 'room_laundry_room', 'room_living_room', 'activity_away', 'activity_cooking', 'activity_idle', 'activity_sleeping', 'activity_watching_tv', 'energy_lag_1H', 'energy_lag_1D', 'energy_lag_1W', 'energy_roll_mean_1hr', 'energy_roll_mean_12hr', 'energy_roll_mean_24hr']

X = X.reindex(columns=final_col_order)

sample_df = pd.DataFrame(X)

scaler = joblib.load('../saved_objects/scaler.joblib')
sample_df[num_cols] = pd.DataFrame(scaler.fit_transform(sample_df[num_cols]), columns=num_cols, index=sample_df.index)
sample_df = sample_df.sort_values(['home_id', 'timestamp'])

In [4]:
drop_cols = ['home_id', 'device_type', 'timestamp']

X_vals = sample_df.drop(columns=drop_cols).values
X_seq = X_vals.reshape(X_vals.shape[0], 1, X_vals.shape[1])

print(X_seq.shape)

(29760, 1, 30)


In [5]:
@tf.keras.utils.register_keras_serializable()
def asymmetric_huber(
    y_true,
    y_pred,
    delta=0.5,
    under_weight=2.2,
    over_weight=1.2
):
    error = y_pred - y_true
    abs_error = tf.abs(error)

    huber = tf.where(
        abs_error <= delta,
        0.5 * tf.square(error),
        delta * (abs_error - 0.5 * delta)
    )

    weight = tf.where(error < 0, under_weight, over_weight)
    return tf.reduce_mean(weight * huber)

In [6]:
mod = load_model('../saved_objects/FinalModel.keras')

y_pred = mod.predict(X_seq)
y_pred[y_pred < 0.02] = 0.0

new_df['predicted_kWh']=y_pred.flatten()
new_df

930/930 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step


,home_id,timestamp,device_type,room,status,user_present,activity,indoor_temp,outdoor_temp,humidity,...,day_of_week,hour_of_day,energy_kWh,energy_lag_1H,energy_lag_1D,energy_lag_1W,energy_roll_mean_1hr,energy_roll_mean_12hr,energy_roll_mean_24hr,predicted_kWh
0,1,2022-01-01 00:00:00,air_conditioner,bedroom,off,1,sleeping,11.4,11.9,45.2,...,5,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.074496
3,1,2022-01-01 00:00:00,fridge,kitchen,on,1,sleeping,11.4,11.9,45.2,...,5,0,0.055865,NaN,NaN,NaN,NaN,NaN,NaN,0.074496
1,1,2022-01-01 00:00:00,light,living_room,on,1,sleeping,11.4,11.9,45.2,...,5,0,0.026470,NaN,NaN,NaN,NaN,NaN,NaN,0.074496
2,1,2022-01-01 00:00:00,tv,living_room,off,1,sleeping,11.4,11.9,45.2,...,5,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.074496
4,1,2022-01-01 00:00:00,washer,laundry_room,off,1,sleeping,11.4,11.9,45.2,...,5,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.074496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148755,2,2022-01-31 23:45:00,air_conditioner,bedroom,off,1,sleeping,8.2,6.2,100.0,...,0,23,0.000000,0.00000,0.175735,0.000000,0.000000,0.003795,0.011808,0.055353
148758,2,2022-01-31 23:45:00,fridge,kitchen,on,1,sleeping,8.2,6.2,100.0,...,0,23,0.067795,0.06668,0.071118,0.052577,0.066529,0.067716,0.067760,0.082260
148756,2,2022-01-31 23:45:00,light,living_room,off,1,sleeping,8.2,6.2,100.0,...,0,23,0.000000,0.00000,0.000000,0.000000,0.012414,0.018452,0.013177,0.000000
148757,2,2022-01-31 23:45:00,tv,living_room,off,1,sleeping,8.2,6.2,100.0,...,0,23,0.000000,0.00000,0.000000,0.000000,0.025548,0.054795,0.040605,0.000000


In [7]:
new_df.to_csv('../data/sample_inputs.csv', index=False)

In [9]:
sample_df.dtypes

home_id                                 int64
device_type                            object
timestamp                      datetime64[ns]
user_present                            int64
status                                  int64
is_weekend                              int64
hour_of_day                             int64
day_of_week                             int64
month_of_year                           int32
indoor_temp                           float64
outdoor_temp                          float64
humidity                              float64
light_level                           float64
device_type_air_conditioner           float64
device_type_fridge                    float64
device_type_light                     float64
device_type_tv                        float64
device_type_washer                    float64
room_bedroom                          float64
room_kitchen                          float64
room_laundry_room                     float64
room_living_room                  